# train_df_icdar

In [ ]:
OFFICIAL RELEASE: icdar_train_df_20250514_175905.csv

Code explanation: prendo i filenames ed estraggo same_text, writer e isEng. Creo un dataframe da questi che contiene anche il percorso del file.
Faccio il merge su writer con il dataframe da train_answers che ha writer e male come colonne.

In [ ]:
Setto un seed -> i risultati dovrebbero essere sempre gli stessi quando seleziono train e val ->
ho verificato che se lancio due volte in momenti diversi ottengo la stessa lista di writers in validation

X
sto prendendo writer, isEng etc dal nome del file e sto associando con le labels
prendo le labels da train_answers.csv

Problema: potrebbe essere che train_answers e train abbiano gli stessi scrittori  (handcrafted features da train è associato correttamente alle labels) 
ma che train_answers e i filenames delle immagini non seguano lo stesso ordinamento degli scrittori?
Come posso verificare che sto associando le immagini corrette?
Posso addestrare un modello che usi cnn. Se efficace non ci dovrebbero essere errori
posso fare un modello che matcha immagini e features
controllo da dove sono scaricati i dati
posso riestrarre le features e compararle con quelle già estratte

problema: potrebeb essere che train answers e train abbiano scrittori ordinati diversamente? Improbabile perchè sono tutti e due dataset di kaggle e FE funziona bene (80% val accuracy)



train_answers 

contiene 282 scrittori.
Number of times 'male' is 0: 143
Number of times 'male' is 1: 139

Dati estratti dai filenames

I load the images from the following folders: ['1_50', '51_100', '101_150', '151_200', '201_250', '251_300']

Costruzione del df da file:
Ho controllato che l'associazione con isEng sia corretta: pg 1 e 2 per uno scrittore sono in arabo, pg 3 e 4 in inglese

Come verificare se sto prendendo il valore giusto? Posso guardare degli esempi: 
-per writer=1 i primi due sono arabi isEng=0 e i secondi due sono inglesi isEng=1. Per writer 1 e writer 2 v=4 ha lo stesso testo mentre v=3 ha testo diverso. Per l'arabo v=2 hanno testo uguale (mi sembra) mentre v=1 ha testo diverso
-lo stesso vale per writer 253 e 254
-> sono ragionevolmente sicuro che sia tutto a posto

- Devo verificare che sto associando i percorsi giusti (che effettivamente nel df che creo writer e same_page siano costruiti nel modo giusto dal filename)
-> guardo il nome dei file e vedo se le colonne corrispondenti (writer, isEng etc ..) hanno il valore corretto. 
Ho controllato per 10 righe prese a caso e l'associazione è corretta

Ho aggiunto la colonna train
I need to check that 90% of writers is with train=1 -> the fractionof row with train=1 is exactly 90%
I need to check that a writer is only in one group -> i counted the unique values for each group of rows that share a writer. I have always one unique value 

Associating with the sex_df (adding the sex column)
- to confirm that the merging is working i randomly sample rows from sex_df and train_df and check that the writer and sex columns are shared -> confirmed

Test di verifica sul dataset finale ->
Number of times 'male' is 0: 572
Number of times 'male' is 1: 556
Number of times 'male' is something else: 0
All writers have both same_text=1 and same_text=0.
All writers have both isEng=1 and isEng=0.
Number of rows where train == 1: 1012
Fraction of rows where train == 1: 0.90
The train column is constant for all writers.
Each unique writer value occurs on exactly 4 rows.

I checked that there are no missing values:
All writers have both same_text=1 and same_text=0.
All writers have both isEng=1 and isEng=0.

ho verificato di avere 4 esempi per ogni scrittore

Prima di lanciare il codice ho fatto push su github
Salvo con seed e descrizione del dataset in un log file di tipo json 

# train_df_icdar_FE

OFFICIAL RELEASE: icdar_train_df_KAGGLE_20250514_181737.csv

Costruisce il df che posso usare per addestrare i modelli che usano le features di kaggle

Semplicemente fa il merging tra il train e il train_answer_df

# feature_extraction

RESULTS:
the validation accuracy I obtain by simply applying gbm to the kaggle features is 75% (100% on training). 
the validation on isEng is 98% -> task is much easier


In [ ]:
In this code I load the dataset generated by train_df_icdar_FE and apply without preprocessing some models like xgb etc ..

I simply need to check that the dataset has non nans, missing values etc .. -> checked and is allright

# patches_cc

In [ ]:
OFFICIAL RELEASE: icdar_train_df_patches_20250514_221029.csv

Carico il file generato da train_df_icdar. Per ogni documento singolo genero delle patches. Genero un nuovo dataframe identico a quello di partenza ma con 
le colonne x,y,x2,y2 per identificare le patches, assegno un valore univoco alle patches per further reference

NOT OPTIMAL
Devo verificare che le patches estratte abbiano senso -> tutte contengono testo ma alcune hanno il campo di vista mezzo vuoto

Devo verificare che ho 5 patch per ogni coppia isEng, same_page fissato lo scrittore
Dato che ogni coppia è associata a un indice unico controllo che per ogni indice ho 5 patches -> ok!

Devo verificare che le nuove patches create mantengano l'associazione con la label e gli altri parametri del df di input
-> estraggo delle righe a caso dal df esteso e vado a confrontare con le righe del writer corrispondente nel df originale

vedo che la riga i-esima che corrisponde ad una patch ha writer=n, male=m e train=t.
Le righe in train_df per writer=n hanno lo stesso m e t -> ragionevolmente sicuro che l'assoicazione è stata 
fatta correttamente

Devo verificare di avere 5 patches per ogni immagine di partenza -> verifico che ogni writer appaia 5*4 volte, che abbia sempre lo stesso valore di male ->ok